In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras import models
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from tensorflow.keras import regularizers


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')


In [ ]:
#'f_12','f_18','f_13','f_16','f_27'


In [ ]:
y_train = train['target']
train = train.drop(['target'],axis=1)
#test = test.drop(['f_12','f_18','f_13','f_16','f_27'],axis=1)


In [ ]:
train.head()


In [ ]:
train = train.set_index('id')
test = test.set_index('id')

train_columns = train.columns
test_columns = test.columns


In [ ]:
scaler = StandardScaler()
le = preprocessing.LabelEncoder()

f_27 = le.fit_transform(train['f_27'])
f_27_test = le.fit_transform(test['f_27'])

train['f_27'] = f_27
test['f_27'] = f_27_test

train = scaler.fit_transform(train)
test = scaler.fit_transform(test)


In [ ]:
train = pd.DataFrame(data=train,columns=train_columns)
test = pd.DataFrame(data=test,columns=test_columns)


In [ ]:
train.head()


In [ ]:
train.shape


In [ ]:
model2 = models.Sequential()
model2.add(layers.Dense(1000, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu",input_shape=(31,)))
model2.add(layers.BatchNormalization())
model2.add(layers.Dense(600, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.1))
model2.add(layers.Dense(200,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.2))
model2.add(layers.Dense(50,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.3))
model2.add(layers.Dense(16,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dense(1,activation='sigmoid'))

model2.summary()


In [ ]:
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model2.fit(train, y_train, validation_split=0.3, shuffle=True, epochs=200,batch_size=2048,callbacks=[callback])


In [ ]:
#model1 = models.Sequential() 
#model1.add(layers.Dense(5000, activation='relu', input_shape=(31,)))
#model1.add(layers.Dropout(0.3))
#model1.add(layers.Dense(600, activation='relu'))
#model1.add(layers.Dense(350, activation='relu'))
#model1.add(layers.Dense(200, activation='relu'))
#model1.add(layers.Dropout(0.3))
#model1.add(layers.Dense(50, activation='relu'))
#model1.add(layers.Dense(15, activation='relu'))
#model1.add(layers.Dense(1, activation='sigmoid')) 

#model1.summary()


In [ ]:
#model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])


In [ ]:
#model1.fit(train, y_train, validation_split=0.3, shuffle=True, epochs=10)


In [ ]:
preds = model2.predict(test)


In [ ]:
sample_sub['target'] = preds


In [ ]:
sample_sub.to_csv('submission.csv', index=False)
